Author: Ruoyu Liu, Yikai Hao, Yu-Chieh Chen

# Overview


Nowadays, people cannot live without their cellphones. People store their important information on their smartphones and use their cellphones for daily communication. The Android Operating System takes over 80% market share of smartphones' operating system. However, as an open source market, about 20% of applications in the market are considered as malicious applications. Those apps will track the user's data, send messages unconsciously, or even lock the user's smartphone until the user pays the ransom.

Due to the seriousness of the malware problem, detecting methods become an expanding research field in academic circles. One most popular detecting method is called HinDroid. It checks the relationships between applications using Heterogeneous Information Network (HIN). To further improve the HinDroid, we plan to design a new model based on the idea of HinDroid and HIN. Meanwhile, we will also implement other models like Metapath2vec and Node2vec in order to evaluate our new model.


# Project Description

## Problem statement and Previous work

As an open source market, Android is often targeted by hackers to build malicious applications. HinDroid uses Heterogeneous Information Network (HIN) to find the similarities between applications and detect malwares. HinDroid design matrices A, B, P, and I, which each matrix represent a kind of relationship between apis or apps by using 1/0. For instance, A matrix represents whether an API is in an app or not. HinDroid then develops different kernels and uses the meta paths between apps to check relationships among apps.

In this quarter’s study, we replicate HinDroid. We start from feature extraction and then build our own A, B, and P matrices based on the Android Malware Dataset (AMD). Due to the complexity of the variables we extract and the size of the features, we design a database to store the information. We also try multiple kernels like $AA^{T}$, $ABA^{T}$, $APA^{T}$, and $APBP^{T}A^{T}$ to make predictions. 

During the implementation of HinDroid, we find some constraints of HinDroid. For example, A matrix only uses 1/0 to represent whether an api is in an app. However, an api might be called multiple times in an app and those apis should be considered more important than others. In addition, when we try kernels with the B matrix, due to the large size of the B matrix, lots of kernels containing the B matrix will take longer computational time and even lead to kernel death. 

## Goals and Deliverables 

Currently, we receive 96.77% on testing accuracy when using the $AA^{T}$ kernel. However, the recall of it is still 0.9655. In this project, our goal will be finding a better model to represent the relationships between applications and locate the exact part of the code which takes the malicious action. We want to improve current HinDroid and build a more robust model for detecting malwares. By adding new features, designing new kernels, and using other models, we want to build a new model with faster speed and higher accuracy. The outcome will be a paper which describes the new model and compares different models.


## Dataset

The dataset we use is AnDroid Malware Dataset (AMD), which is published in 2017 by the Argus Lab from the University of South Florida. Many other research papers about malwares are also based on the same dataset. It is an appropriate dataset to use to build and analyze our model.

However, as a quickly changing area, hackers might use different methods to avoid detections in small times. Therefore, since the dataset is from 2017, we might need to consider adding some more recent data from reliable sources.


# Project Methods

Our process of developing the new model will be mainly separated into four parts.

## New Features

On top of HinDroid, which extracts api call, code block, package name, and invoke type as features, we would like to add more features into the heterogeneous map. For example, the `class type` and `method type` might also give us some hints. For example, the class and method can be private or public. The `return type` can also help us identify what is the function doing and the outcome of the function. More research is also needed for this part to help us understand how the hackers will put malicious action inside the code and how we should select that part out.

A sample of our new database is listed below.

![new_database.png](../data/report/newdatabase.png)

### Class Type

We summarize the class into three different upper level types - public, default, and private. All three types of applications do not contain private class types and have more public class types than default class types. However, malware’s default class types take more percentage than random and popular applications.  

As we can see from the graph, the distribution of class among three types of applications is different. More specifically, malwares only show up a lot in class types public, public final, default, and final with a small count. Popular and random applications have a more wide spread class type distribution.


|   | Malware |   Random  | Popular | 
|:--------:|:--------:|:------:|:------:|
| Public | 0.6332  | 0.7295 | 0.7216 |
| Default |  0.3668  | 0.2705 | 0.2784 |
| Private |  0 | 0 | 0  |

![class.png](../data/report/class_proposal.png)

Note: class_id's corresponding class_type name is shown in Appendix table

### Method Type

Similar to class type, we also separate method types into 4 main categories - `public`, `private`, `protected`, and `other`. As we can see from the table, the percentage each type of application takes is similar in all 4 categories. Therefore, we will do a closer analysis on this next quarter. 

We then select out the top 20 method types that are called in smali files. As shown in the graph below, the distributions have some differences among three types of applications. The mean count of malwares are always smaller than other two types. 


|   | Malware |   Random  | Popular | 
|:--------:|:--------:|:------:|:------:|
| Public | 0.584849  | 0.6173 | 0.6181 |
| Private |  0.193660 | 0.1905 | 0.2044  |
| Protected |  0.124208 | 0.1317 | 0.1242 |
| Other |  0.097283 | 0.0606 | 0.0532  |

![method.png](../data/report/method_proposal.png)

Note: method_id's corresponding method_type name is shown in Appendix table

### Return Type

In the next quarter, our group plans to add return type as a new feature. Currently, we investigated the family of the API's return type. In the below table, we realized that `Ljava` families are used significantly across all the categories of apps. Besides this, the top four families of the return type are the same across three categories of apps. As we see in the table, the fifth common family in random and popular apps, `Landroidx`, is never presenting in malware apps. This requires further investigation. 

#### Top family for return type 
|   | Malware |   Random  | Popular | 
|:--------:|:--------:|:------:|:------:|
| Ljava | 0.6554 (#1) | 0.6008 (#1) | 0.6182 (#1) |
| Landroid |  0.2415 (#2) | 0.2068 (#2) | 0.1649 (#2)  |
| Lcom |  0.0398 (#3) | 0.1520 (#3) | 0.1523 (#3) |
| Lorg |  0.0382 (#4) | 0.0154 (#4) | 0.0181 (#4)  |
| Lnet |  0.0095 (#5) | 0.0002 (#20) | 0.01341 (#6)  |
| Landroidx | Not in Data | 0.0054 (#5) | 0.0134 (#5)  |

## New Kernel

Our goal is to continue replicating the HinDroid paper. Currently, we have implemented $AA^{T}$, $ABA^{T}$, $APA^{T}$, and $APBP^{T}A^{T}$ kernels. In the next quarter, we would like to implement $ABPB^{T}A^{T}$ and $AIA^{T}$ kernels. In addition, we might add some new matrices which takes the times an api in an app is called into account. 

### $ABPB^{T}A^{T}$


![ABPBA](../data/report/ABPBA.jpg)

The meta-path is APP $\xrightarrow[]{\text{contains}}$ API $\xrightarrow[]{\text{same code block}}$API $\xrightarrow[]{\text{same package}}$API$\xrightarrow[]{\text{ $same$ $code$ $block^{-1}$}}$API$\xrightarrow[]{\text{$contains^{-1}$}}$ APP 

The items in $ABPB^{T}A^{T}$ the number of paired apis used in the same package that are in the same code block between two apps.

### $AIA^{T}$

![AIA](../data/report/AIA.jpg)


The meta-path is APP $\xrightarrow[]{\text{contains}}$ API  $\xrightarrow[]{\text{same invoke type}}$API$\xrightarrow[]{\text{$contains^{-1}$}}$ APP 

The items in $AIA^{T}$ represent the number of paired apis in the using the same invoke type between two apps.


### Weighted Graph 

New kernel will contain matrices that add weight to different apis. For example, if the api occurs in an application 10 times, the new aij will be 10 instead of 1. Since there might contain really large numbers, we might need to design a new way to add weight instead of directly changing the 1/0 to count.


## New Model

### Node2Vec

Node2Vec is based on Word2Vec with a different sampling strategy. Node2Vec generates corpus using the homogeneous graph which represents all the edges and nodes as the same type.  Its sampling strategy is controlled by 4 hyperparameters: number of walks, walk length, p, and q. 


### Metapath2Vec

Metapath2Vec differs from the two above methods because it uses heterogeneous graphs, which means that the data nodes can be different types. Unlike Node2Vec, Metapath2Vec’s sampling strategy is primarily controlled by the input Metapath. Due to the reason that HinDroid creates specific kernels which can be used as metapath, we think Metapath2Vec should have better performance compared to Node2Vec. 

### Other Models

Besides the above different learning methods, our group plans to change the current SVM model to other machine learning or deep learning methods, like CNN and RNN. 


## Other Methods
In this quarter, we are facing a kernel dead issue when running on $ABPB^{T}A^{T}$ kernel. The reason behind this problem is there are too many apis. Since the B matrix is in dimension the number of APIs times the number of APIs, the B matrix will be really large. Currently, the B matrix contains 2177789 x 2177789 columns, which represent that there are 2177789 unique api calls in total 742 apps. Since there are lots of APIs occurring in almost every application and some APIs only occur in a few applications, we want to select out most representative APIs which can represent whether the application is malware or not.

# Evaluation

As a project aimed at detecting malwares, the first evaluation we will make on our new model is the accuracy. Besides picking out malwares, we also do not want to tag benign apps as malwares. Therefore, precision and recall are both important for us. We will use the F1 score as a reference to check our new model.
Since hackers keep developing new methods to avoid detection, we also need to check our model with datas from different times, especially the most recent data. When making comparisons, we also want to compare their performance based on different years since we do not want our outcome to be useful in a few years.
 Graphs are also important for our models. Since HinDroid, Metapath2vec, Node2vec, Word2vec, and our new model are all based on the idea of graphs to check the relationship between apps, it is important to try to plot the graph to show the algorithm behind each model. Therefore, we will also try to compare different models based on the graph they create.


# Schedule 

![schedule](../data/report/schedule.png)


# Reference

[1]: "HinDroid: An Intelligent Android Malware Detection System Based on Structured Heterogeneous Information Network" https://www.cse.ust.hk/~yqsong/papers/2017-KDD-HINDROID.pdf

[2]: "Efficient Estimation of Word Representations in Vector Space" https://arxiv.org/pdf/1301.3781.pdf

[3]: "node2vec: Scalable Feature Learning for Networks" https://arxiv.org/pdf/1607.00653.pdf

[4]: "metapath2vec: Scalable Representation Learning for Heterogeneous Networks" https://ericdongyx.github.io/papers/KDD17-dong-chawla-swami-metapath2vec.pdf


# Appendix

- Class_type vs. Class_type_id

| class_type_id | class_type |
|:--------:|:--------:|
| 0 | public |
| 1 | public final |
| 2 | default |
| 3 | public interface abstract |
| 4 | final |
| 5 | interface abstract |
| 6 | public abstract |
| 7 | synthetic |
| 8 | abstract |
| 9 | public final enum |
| 10 | final enum |
| 11 | interface |
| 12 | enum |
| 13 | public abstract enum |
| 14 | public interface abstract annotation |
| 15 | public enum |
| 16 | interface abstract annotation |
| 17 | final synthetic |
| 18 | abstract enum |
| 19 | public abstract synthetic |
| 20 | public synthetic |
| 21 | public final synthetic |

- Method_id vs. Method_type

| method_id | method type|
|------:|:------:|
| 0 | public constructor |
| 1 | public |
| 2 | protected |
| 3 | private |
| 6 | constructor |
| 7 | private constructor |
| 9 | public static |
| 10 | private static |
| 13 | public final |
| 14 | static constructor |
| 15 | public bridge synthetic |
| 16 | static |
| 18 | default |
| 24 | final |
| 29 | protected final |
| 30 | public static final |
| 39 | public declared-synchronized |
| 44 | public synthetic |
| 47 | public final synthetic |
| 53 | private final |
